In [1]:
# 1. Ler os CSVs brutos da pasta bronze
df_raw = spark.read.option("header", True).option("sep", ";").option("encoding", "latin1") \
    .csv("Files/bronze/fi_m-2-m-11_meses/*.csv")

StatementMeta(, 8037f443-f7f4-4792-b2a9-ed23fffe616d, 3, Finished, Available, Finished)

In [2]:
# 2. Criar uma view temporária para rodar SQL sobre esse DataFrame
df_raw.createOrReplaceTempView("vw_inf_diario_raw")

StatementMeta(, 8037f443-f7f4-4792-b2a9-ed23fffe616d, 4, Finished, Available, Finished)

In [3]:
# 3. Aplicar SQL de transformação com CASTs e limpeza
# Ja apagando a coluna de ID_SUBCLASSE
df_limp = spark.sql("""    
    SELECT
        TP_FUNDO_CLASSE,
        CNPJ_FUNDO_CLASSE,
        CAST(DT_COMPTC AS DATE) AS DT_COMPTC,
        CAST(REPLACE(VL_QUOTA, ',', '.') AS DOUBLE) AS VL_QUOTA,
        CAST(REPLACE(VL_PATRIM_LIQ, ',', '.') AS DOUBLE) AS VL_PATRIM_LIQ,
        CAST(REPLACE(CAPTC_DIA, ',', '.') AS DOUBLE) AS CAPTC_DIA,
        CAST(REPLACE(RESG_DIA, ',', '.') AS DOUBLE) AS RESG_DIA,
        CAST(REPLACE(NR_COTST, ',', '.') AS DOUBLE) AS NR_COTST
    FROM vw_inf_diario_raw
    WHERE VL_QUOTA IS NOT NULL AND DT_COMPTC IS NOT NULL
""")

StatementMeta(, 8037f443-f7f4-4792-b2a9-ed23fffe616d, 5, Finished, Available, Finished)

In [4]:
df_limp.printSchema()
display(df_limp)


StatementMeta(, 8037f443-f7f4-4792-b2a9-ed23fffe616d, 6, Finished, Available, Finished)

root
 |-- TP_FUNDO_CLASSE: string (nullable = true)
 |-- CNPJ_FUNDO_CLASSE: string (nullable = true)
 |-- DT_COMPTC: date (nullable = true)
 |-- VL_QUOTA: double (nullable = true)
 |-- VL_PATRIM_LIQ: double (nullable = true)
 |-- CAPTC_DIA: double (nullable = true)
 |-- RESG_DIA: double (nullable = true)
 |-- NR_COTST: double (nullable = true)



SynapseWidget(Synapse.DataFrame, 912175c6-ad9b-4e4d-aad0-5a7e27cd74db)

In [5]:
# 4. Salvar em parquet na camada silver
df_limp.write.mode("overwrite").parquet("Files/silver/fi_m-2-m-11_meses")

StatementMeta(, 8037f443-f7f4-4792-b2a9-ed23fffe616d, 7, Finished, Available, Finished)